#### Enter the Workspace Name & Lakehouse Name where Shortcuts are located


In [1]:
Workspace = "426957d6-d878-4f19-9129-eddb54d242df"
Lakehouse = "24efbf99-0d02-4bc2-aa17-39346211de99"
input_name = "DummyData"  # <-- Replace your table/shortcut name to be checked

In [2]:
%run NB - Load Configuration

#### Listing Shortcut Tables

In [3]:
import requests
import pandas as pd

def fetch_shortcut_names(workspace_id, lakehouse_id):
    resource = "https://api.fabric.microsoft.com"
    token = mssparkutils.credentials.getToken(resource)
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    shortcuts_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts"
    
    print("\nFetching Shortcuts...")
    response = requests.get(shortcuts_url, headers=headers)

    if response.ok:
        shortcut_items = response.json().get("value", [])
        if shortcut_items:
            shortcut_names_df = pd.DataFrame([item["name"] for item in shortcut_items if "name" in item], columns=["name"])
            return shortcut_names_df
        else:
            print("No shortcuts found.")
            return pd.DataFrame(columns=["name"])
    else:
        print(f"Error fetching shortcuts: {response.status_code} - {response.text}")
        return pd.DataFrame(columns=["name"])

try:
    if not is_parent_call:  # is_parent_call 
        result_df = fetch_shortcut_names(workspace_id, lakehouse_id)
        display(result_df)
except NameError:
    #  standalone
    workspace_id = Workspace
    lakehouse_id = Lakehouse
    result_df = fetch_shortcut_names(workspace_id, lakehouse_id)
    display(result_df)


##### Checking whether a given table name is a shortcut table or not

In [4]:
import requests

def check_if_shortcut(workspace_id, lakehouse_id, input_name):
    resource = "https://api.fabric.microsoft.com"
    token = mssparkutils.credentials.getToken(resource)

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    shortcuts_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts"
    shortcuts_response = requests.get(shortcuts_url, headers=headers)

    is_shortcut = False

    if shortcuts_response.ok:
        shortcut_items = shortcuts_response.json().get("value", [])
        for item in shortcut_items:
            shortcut_name = item.get("name", "").strip().lower()
            if shortcut_name == input_name.strip().lower():
                is_shortcut = True
                break

        if is_shortcut:
            print(f"{input_name} is a Shortcut table.")
        else:
            print(f"{input_name} is NOT a Shortcut table.")
    else:
        print(f"Error fetching shortcuts: {shortcuts_response.status_code} - {shortcuts_response.text}")

    # Optional return 
    if not is_parent_call:
        return is_shortcut

# Safely handle undefined 'is_parent_call'
try:
    is_parent_call
except NameError:
    is_parent_call = False

#  Trigger check if not called by parent
if not is_parent_call:
    check_if_shortcut(Workspace, Lakehouse, input_name)
